In [1]:
import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
Independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [9]:
Independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
Dependent=dataset[["Purchased"]]

In [11]:
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [12]:
Independent.shape

(400, 3)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'kernel':('linear','poly','sigmoid','rbf'),'gamma':('scale','auto'),'C':(10,100,1000,2000,3000)}
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,cv=5)

In [16]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\91805\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': (10, 100, 1000, 2000, 3000),
                         'gamma': ('scale', 'auto'),
                         'kernel': ('linear', 'poly', 'sigmoid', 'rbf')},
             verbose=3)

In [17]:
y_pred=grid.predict(x_test)

In [18]:
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report

In [19]:
cm=confusion_matrix(y_test,y_pred)

In [20]:
print(cm)

[[73  6]
 [ 4 37]]


In [21]:
clf_rpt=classification_report(y_test,y_pred)

In [22]:
print(clf_rpt)

              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [23]:
roc_score=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

In [24]:
roc_score

0.9654214263661625

In [25]:
re=grid.cv_results_

In [26]:
print("The best parameters is" ,grid.best_params_)

The best parameters is {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [27]:
table=pd.DataFrame.from_dict(re)

In [28]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027309,0.004923,0.007113,0.002037,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.839286,0.785714,0.678571,0.928571,0.892857,0.825000,0.087773,21
1,0.028526,0.006555,0.006325,0.000766,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.839286,0.803571,0.750000,0.928571,0.892857,0.842857,0.063286,12
2,0.017282,0.002900,0.006339,0.001791,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.767857,0.732143,0.625000,0.750000,0.767857,0.728571,0.053452,40
3,0.017781,0.004471,0.007439,0.001716,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875000,0.839286,0.946429,0.946429,0.900000,0.041650,1
4,0.021480,0.002721,0.006256,0.002340,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.839286,0.785714,0.678571,0.928571,0.892857,0.825000,0.087773,21
5,0.023115,0.002465,0.006290,0.002289,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.839286,0.803571,0.750000,0.928571,0.892857,0.842857,0.063286,12
6,0.015127,0.002528,0.006800,0.001427,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.767857,0.732143,0.732143,0.750000,0.767857,0.750000,0.015972,31
7,0.025244,0.005176,0.005773,0.001277,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875000,0.839286,0.946429,0.946429,0.900000,0.041650,1
8,0.061642,0.003246,0.004069,0.000421,100,scale,linear,"{'C': 100, 'gamma': 'scale', 'kernel': 'linear'}",0.839286,0.785714,0.678571,0.928571,0.892857,0.825000,0.087773,21
9,0.070618,0.016838,0.006822,0.002559,100,scale,poly,"{'C': 100, 'gamma': 'scale', 'kernel': 'poly'}",0.839286,0.803571,0.750000,0.928571,0.910714,0.846429,0.066432,11


In [64]:
filename="Best classification model with split data using Grid.sav"

In [66]:
import pickle
pickle.dump(grid,open(filename,'wb'))

In [68]:
loaded_model=pickle.load(open("Best classification model with split data using Grid.sav",'rb'))

In [72]:
Age=int(input("Enter you age:"))
Estimatedsalary=int(input("Enter you estimated salary:"))
Gender_Male=int(input("Enter 0 if male or else1:"))


Enter you age: 27
Enter you estimated salary: 68000
Enter 0 if male or else1: 1


In [76]:
result=loaded_model.predict([[Age,Estimatedsalary,Gender_Male]])

In [78]:
result

array([1], dtype=int64)